In [1]:
# !pip install -U accelerate peft bitsandbytes transformers trl datasets

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4,"

In [3]:
import torch
from datasets import load_dataset
from transformers import (
    TrainingArguments,
    AutoModelForCausalLM,
    AutoTokenizer,
)
from trl import SFTTrainer

In [4]:
out_dir = "./saved_models/kogpt2_koalpaca"
model_name = "skt/kogpt2-base-v2"
batch_size = 8
num_train_epochs = 5
logging_steps = 500
bf16 = False
fp16 = True
context_length = 256
num_workers = 1
# num_workers = os.cpu_count()
gradient_accumulation_steps = 2
learning_rate = 0.0001

In [5]:
# Load dataset
dataset = load_dataset('bingsu/ko_alpaca_data')

In [6]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 49620
    })
})


In [7]:
print(dataset['train']['instruction'][0])

건강을 유지하기 위한 세 가지 팁을 알려주세요.


In [8]:
print(dataset['train'][0])

{'instruction': '건강을 유지하기 위한 세 가지 팁을 알려주세요.', 'input': '', 'output': '세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.'}


In [9]:
# Create train set and validation set
full_dataset = dataset['train'].train_test_split(test_size=0.05, shuffle=True)
dataset_train = full_dataset['train']
dataset_valid = full_dataset['test']

print(f"Train set size: {len(dataset_train)}, Valid set size: {len(dataset_valid)}")

Train set size: 47139, Valid set size: 2481


In [10]:
# Check dataset as alpaca prompt format
for i in range(3):
    print(dataset_train[i])
    print('****************')
    
    text = dataset_train[i]
    instruction = '### Instruction:\n' + text['instruction']
    inputs = '\n\n### Input:\n' + text['input']
    response = '\n\n### Response:\n' + text['output']
    
    final_text = instruction + inputs + response
    print(final_text)
    print('#'*50)

{'instruction': '균형 잡힌 식단의 다섯 가지 구성 요소를 말하십시오.', 'input': '', 'output': '균형 잡힌 식단에는 곡류, 채소, 단백질, 유지 및 과일이 포함됩니다.'}
****************
### Instruction:
균형 잡힌 식단의 다섯 가지 구성 요소를 말하십시오.

### Input:


### Response:
균형 잡힌 식단에는 곡류, 채소, 단백질, 유지 및 과일이 포함됩니다.
##################################################
{'instruction': '웹사이트의 신뢰성 여부를 판단하는 알고리즘을 만드세요.', 'input': '', 'output': '웹사이트 신뢰성 판단의 일반적인 방법 중 하나는 해당 웹사이트의 SSL/TLS 인증서 여부입니다. SSL/TLS 인증서가 있는 경우, 정보가 암호화되고 안전한 연결이 보장됩니다. 또는 해당 웹사이트가 보유한 개인 정보, 역사, 평판 및 기타 정보를 확인하여 신뢰할 수 있는지 여부를 결정할 수도 있습니다.'}
****************
### Instruction:
웹사이트의 신뢰성 여부를 판단하는 알고리즘을 만드세요.

### Input:


### Response:
웹사이트 신뢰성 판단의 일반적인 방법 중 하나는 해당 웹사이트의 SSL/TLS 인증서 여부입니다. SSL/TLS 인증서가 있는 경우, 정보가 암호화되고 안전한 연결이 보장됩니다. 또는 해당 웹사이트가 보유한 개인 정보, 역사, 평판 및 기타 정보를 확인하여 신뢰할 수 있는지 여부를 결정할 수도 있습니다.
##################################################
{'instruction': '선택한 캐릭터에 대한 짧은 이야기를 작성합니다.', 'input': '캐릭터: 여우 아델', 'output': '옛날 옛적에 여우 아델이 울길 많은 날이었습니다.'}
****************
### Instructi

In [11]:
# Mapping function for dataset
def preprocess_function(examples):
    """
    Formatting function returning a list of processed strings.
    """
    texts = []
    
    for example in zip(examples['instruction'], examples['input'], examples['output']):
        instruction, input_text, output = example
        text = f"### 지시:\n{instruction}\n\n### 자료:\n{input_text}\n\n### 응답:\n{output}"
        texts.append(text)
    
    return texts

In [12]:
# Load model
if bf16:
    model = AutoModelForCausalLM.from_pretrained(model_name).to(dtype=torch.bfloat16)
else:
    model = AutoModelForCausalLM.from_pretrained(model_name)

In [13]:
# Check model
print(model)
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)
125,164,032 total parameters.
125,164,032 training parameters.


In [14]:
# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_name, 
    trust_remote_code=True,
    use_fast=True,
)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '</s>', 'eos_token':'</s>'})
    model.resize_token_embeddings(len(tokenizer))

In [15]:
# Set training arguments
training_args = TrainingArguments(
    output_dir=f"{out_dir}/logs",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    eval_strategy='epoch',
    num_train_epochs=num_train_epochs,
    logging_strategy='steps',
    logging_steps=logging_steps,
    save_strategy='epoch',
    save_total_limit=2,
    load_best_model_at_end=True,
    bf16=bf16,
    fp16=fp16,
    weight_decay=0.01,
    report_to='tensorboard',
    dataloader_num_workers=num_workers,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    lr_scheduler_type='constant',
)

In [16]:
# Load trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_train,
    eval_dataset=dataset_valid,
    max_seq_length=context_length,
    tokenizer=tokenizer,
    args=training_args,
    formatting_func=preprocess_function,
    # packing=True,
)

/home/sslunder13/env/transformers/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/sslunder13/env/transformers/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/47139 [00:00<?, ? examples/s]

Map:   0%|          | 0/2481 [00:00<?, ? examples/s]

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [17]:
# Training
history = trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch,Training Loss,Validation Loss
0,2.401500,2.309197
2,1.883500,2.287354
4,1.507000,2.403161


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [18]:
# Save model
model.save_pretrained(out_dir)
tokenizer.save_pretrained(out_dir)

('./saved_models/kogpt2_koalpaca/tokenizer_config.json',
 './saved_models/kogpt2_koalpaca/special_tokens_map.json',
 './saved_models/kogpt2_koalpaca/vocab.json',
 './saved_models/kogpt2_koalpaca/merges.txt',
 './saved_models/kogpt2_koalpaca/added_tokens.json',
 './saved_models/kogpt2_koalpaca/tokenizer.json')

In [19]:
from transformers import (
    AutoModelForCausalLM, 
    logging, 
    pipeline,
    AutoTokenizer
)

In [20]:
# Load newly trained model
model = AutoModelForCausalLM.from_pretrained("/home/sslunder13/project/06_instruction_tuning/saved_models/kogpt2_koalpaca/kogpt2_koalpaca")
tokenizer = AutoTokenizer.from_pretrained("/home/sslunder13/project/06_instruction_tuning/saved_models/kogpt2_koalpaca/kogpt2_koalpaca")

tokenizer.pad_token = tokenizer.eos_token

In [29]:
# Inference using fine-tuned model
pipe = pipeline(task='text-generation', model=model, tokenizer=tokenizer, max_length=128)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [30]:
# Input prompt in alpaca format

prompt = """### 지시:
자료에서 느껴지는 감정이 긍정적인지 부정적인지 알려줘.

### 자료:
기분이 최고야!

### 응답:
"""

In [31]:
# Check generated response
result = pipe(prompt)
print(result[0]['generated_text'])

### 지시:
자료에서 느껴지는 감정이 긍정적인지 부정적인지 알려줘.

### 자료:
기분이 최고야!

### 응답:
긍정적인 감정입니다. 기분이 좋은 것으로 생각되어 있습니다. 어떤 감정이든 긍정적일 때가 많습니다. 긍정적인 감정입니다. "그동안은 항상 즐거움으로 인해 매우 즐거운 일이 많았을 것 같아요."를 긍정적인 감정입니다. 그 중 하나는 긍정적인 감정입니다. 기쁨과 안도감, 충동, 만족감 등이 가장 큰 감정입니다. 기분이 좋은 곳으로 생각되던 경험을 통해, 그것은 긍정적인 경험으로 바꿀 수 있었다. 이 일을 하면 기분 좋은 순간입니다. 기분이 좋아
